In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
from datetime import datetime as dt
import sys
import numpy as np
import os
import pandas as pd
from PIL import Image
import plotly
from plotly import subplots
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly, save_plotly_in_en
import requests
from sklearn.linear_model import LinearRegression
from nhk_lib import get_nhk_df, get_nhk_keys, get_lr_col, \
    add_pref2fig, update_axes, update_layout, get_template_fig

In [ ]:
if dt.now().weekday() != 5:
    print("Today is not Saturday.")
    if not "ipy" in sys.argv[0]:
        sys.exit()

In [ ]:
if dt.now().hour < 18:
    print("before 6 pm.")
    if not "ipy" in sys.argv[0]:
        sys.exit()

In [ ]:
today_str = " " + dt.now().isoformat()[:16].replace('T', ' ')

In [ ]:
df = get_nhk_df("nhk.csv")

In [ ]:
k = get_nhk_keys()

In [ ]:
latest = df[k.ts].iloc[-1]

In [ ]:
with open("nhk.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
if latest == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()
with open("nhk.prev.tmp", "wt") as f:
    f.write(latest)

In [ ]:
# 文字列を日付型に変換
df[k.ts] = pd.to_datetime(df[k.ts])

In [ ]:
# 都道府県名を取得
prefs = df[k.pref_name].unique()

In [ ]:
fig1, fig2 = get_template_fig(prefs)

In [ ]:
lr = LinearRegression()

In [ ]:
X = np.arange(14)
for y in np.arange(4):
    for x in np.arange(6):
        i = x + (y * 6)
        add_pref2fig(df, lr, i, x+1, y+1, X, fig1, prefs)
        if i < 23:
            add_pref2fig(df, lr, i + 24, x+1, y+1, X, fig2, prefs)

In [ ]:
update_axes(fig1)
update_axes(fig2)
twbody1 = "新型コロナ 各都道府県 直近2週間状況1"
update_layout(fig1, twbody1, today_str)
twbody2 = "新型コロナ 各都道府県 直近2週間状況2"
update_layout(fig2, twbody2, today_str)

In [ ]:
show_and_save_plotly(fig1, "nhk-all-1.jpg", html=True)

In [ ]:
show_and_save_plotly(fig2, "nhk-all-2.jpg", html=True)

In [ ]:
def vcat_images(x, y, z, dirname=""):
    img1 = Image.open("docs/images/{}{}".format(dirname, x))
    img2 = Image.open("docs/images/{}{}".format(dirname, y))
    img = Image.new('RGB', (img1.width, img1.height + img2.height))
    img.paste(img1, (0, 0))
    img.paste(img2, (0, img1.height))
    img.save("docs/images/{}{}".format(dirname, z))

In [ ]:
vcat_images("nhk-all-1.jpg", "nhk-all-2.jpg", "nhk-all-a.jpg")

In [ ]:
tw = get_twitter()
notes = " (縦軸範囲は各都道府県の最大値) https://geneasyura.github.io/cov19-hm/nhk-1-2.html "
twbody = "新型コロナ 各都道府県 直近2週間感染状況"
tweet_with_image(tw, "docs/images/nhk-all-a.jpg", twbody + today_str + notes)

In [ ]:
def j2e_titles(a, b):
    cnt = 0
    for t in en_prefs['e'].to_list():
        if cnt < 24:
            a['layout']['annotations'][cnt]['text'] = t
        else:
            b['layout']['annotations'][cnt - 24]['text'] = t
        cnt += 1

In [ ]:
en_prefs = pd.read_csv('population.txt', encoding='utf-8', header=None, names=['a', 'b', 'j', 'e', 'p', 'r'])

In [ ]:
j2e_titles(fig1, fig2)

In [ ]:
twbody = "Japan Daily new confirmed COVID-19 cases for last 2 weeks "
fig1['layout']['title']['text'] = twbody
fig2['layout']['title']['text'] = twbody

In [ ]:
save_plotly_in_en(fig1, "nhk-all-1.jpg")
save_plotly_in_en(fig2, "nhk-all-2.jpg")
vcat_images("nhk-all-1.jpg", "nhk-all-2.jpg", "nhk-all-a.jpg", dirname="en/")

In [ ]:
notes = " (the maximum range of y-axis is refered to each prefecture) "
tweet_with_image(tw, "docs/images/en/nhk-all-a.jpg", twbody + today_str + notes)

In [ ]:
def add_trace2fig(dfx, i, x, y, Xx, fig, key, prefs, div=10):
    dfh = dfx[ dfx['都道府県名'] == prefs[i]]
    Y = dfh[key].values
    #trace_xy = go.Scatter(
    #    x=Xx, y=Y, mode='lines', line=dict(width=1, color='white'))
    trace_ma = go.Scatter(
        x=Xx[::div], y=moving_average(Y)[::div], mode='lines',
        line=dict(width=1.25, color='green'))
    #fig.add_trace(trace_xy, y, x)
    fig.add_trace(trace_ma, y, x)

In [ ]:
def make_prefs_subplots(dfx, k, figa, figb, Xx, prefs, div=10):
    for y in np.arange(4):
        for x in np.arange(6):
            i = x + (y * 6)
            add_trace2fig(dfx, i, x+1, y+1, Xx, figa, k, prefs, div)
            if i < 23:
                add_trace2fig(dfx, i + 24, x+1, y+1, Xx, figb, k, prefs, div)

In [ ]:
fig3, fig4 = get_template_fig(prefs)

In [ ]:
xbins = df[k.ts]
make_prefs_subplots(df, k.case, fig3, fig4, xbins, prefs)
update_axes(fig3)
update_axes(fig4)
twbody3 = "新型コロナ 各都道府県 全期間感染状況1"
update_layout(fig3, twbody3, today_str)
twbody4 = "新型コロナ 各都道府県 全期間感染状況2"
update_layout(fig4, twbody4, today_str)

In [ ]:
show_and_save_plotly(fig3, "nhk-all-3.jpg", html=True)

In [ ]:
show_and_save_plotly(fig4, "nhk-all-4.jpg", html=True)

In [ ]:
vcat_images("nhk-all-3.jpg", "nhk-all-4.jpg", "nhk-all-b.jpg")

In [ ]:
twbody = "新型コロナ 各都道府県 全期間感染状況"
notes = " (縦軸範囲は各都道府県の最大値) https://geneasyura.github.io/cov19-hm/nhk-3-4.html "
tweet_with_image(tw, "docs/images/nhk-all-b.jpg", twbody + today_str + notes)

In [ ]:
j2e_titles(fig3, fig4)

In [ ]:
twbody = "Japan Daily new confirmed COVID-19 cases "
fig3['layout']['title']['text'] = twbody
fig4['layout']['title']['text'] = twbody

In [ ]:
save_plotly_in_en(fig3, "nhk-all-3.jpg")
save_plotly_in_en(fig4, "nhk-all-4.jpg")
vcat_images("nhk-all-3.jpg", "nhk-all-4.jpg", "nhk-all-b.jpg", dirname="en/")

In [ ]:
notes = " shown is the rolling 7-day average (the maximum range of y-axis is refered to each prefecture) "
tweet_with_image(tw, "docs/images/en/nhk-all-b.jpg", twbody + today_str + notes)

In [ ]:
fig5, fig6 = get_template_fig(prefs)

In [ ]:
xbins = df[k.ts]
make_prefs_subplots(df, k.death, fig5, fig6, xbins, prefs, div=8)
update_axes(fig5)
update_axes(fig6)
twbody5 = "新型コロナ 各都道府県 全期間死者状況1"
update_layout(fig5, twbody5, today_str)
twbody6 = "新型コロナ 各都道府県 全期間死者状況2"
update_layout(fig6, twbody6, today_str)

In [ ]:
show_and_save_plotly(fig5, "nhk-all-5.jpg", html=True)

In [ ]:
show_and_save_plotly(fig6, "nhk-all-6.jpg", html=True)

In [ ]:
vcat_images("nhk-all-5.jpg", "nhk-all-6.jpg", "nhk-all-c.jpg")

In [ ]:
twbody = "新型コロナ 各都道府県 全期間死者状況"
notes = " (縦軸範囲は各都道府県の最大値) https://geneasyura.github.io/cov19-hm/nhk-5-6.html "
tweet_with_image(tw, "docs/images/nhk-all-c.jpg", twbody + today_str + notes)

In [ ]:
j2e_titles(fig5, fig6)

In [ ]:
twbody = "Japan Daily new confirmed COVID-19 deaths"
fig5['layout']['title']['text'] = twbody
fig6['layout']['title']['text'] = twbody

In [ ]:
save_plotly_in_en(fig5, "nhk-all-5.jpg")
save_plotly_in_en(fig6, "nhk-all-6.jpg")
vcat_images("nhk-all-5.jpg", "nhk-all-6.jpg", "nhk-all-c.jpg", dirname="en/")

In [ ]:
notes = " (the maximum range of y-axis is refered to each prefecture) "
tweet_with_image(tw, "docs/images/en/nhk-all-c.jpg", twbody + today_str + notes)

In [ ]:
#df[ df[k_ts] >= pd.to_datetime("2020-02-01")]

In [ ]:
# 週ごとの集計に変換
df_group_by_w = df[ df[k.ts] >= pd.to_datetime("2020-03-01")].groupby(k.pref_name)\
    .resample('W-Mon', on=k.ts).sum().reset_index().sort_values(by=k.ts)
# 累計値の列を削除
df_group_by_w = df_group_by_w.drop(k.total_case, axis=1)
df_group_by_w = df_group_by_w.drop(k.total_death, axis=1)
# 7 日後を追加
df_group_by_w['p1w'] = df_group_by_w[k.ts] + pd.Timedelta(days=7)

In [ ]:
# int から float へ変換
df_group_by_w = df_group_by_w.astype({k.case: float})
df_group_by_w = df_group_by_w.astype({k.death: float})
#df_group_by_w

In [ ]:
# 都道府県別人口を取得
p = get_populations()
# 都道府県コード
pcodes = {}
# 都道府県人口
ppops = {}
for ky, v in p.items():
    pcodes[v['ja']] = v['code']
    ppops[v['ja']] = v['total']

In [ ]:
# ppm で正規化する
ppm = 1000000
for index, row in df_group_by_w.iterrows():
    pname = row[k.pref_name].rstrip('県').rstrip('都').rstrip('府')
    df_group_by_w.at[index, k.pref_code] = pcodes[pname]
    df_group_by_w.at[index, k.case] = (float(row[k.case]) / ppops[pname]) * ppm
    df_group_by_w.at[index, k.death] = (float(row[k.death]) / ppops[pname]) * ppm

In [ ]:
# データの行を都道府県コード、日付でソート
df_group_by_w = df_group_by_w.reindex(
    columns=[k.pref_code, k.ts, 'p1w', k.case, k.death, k.pref_name])
df_group_by_w = df_group_by_w.sort_values([k.pref_code, k.ts])

In [ ]:
fig7 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_group_by_w, x_start=k.ts, x_end='p1w', y=k.pref_name, color=k.case,
    color_continuous_scale='Rainbow', labels={k.case:'cpm'})
twbody7 = "全国 新型コロナ 都道府県別 週毎 感染者数"
update_layout(fig7, twbody7, today_str)
fig7.update_layout(
    width=800, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig7.update_yaxes(autorange="reversed", side="right")
fig7.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.08, 
        tickfont=dict(color="white"),
        ticklabelposition="outside", ticks="inside"))
show_and_save_plotly(fig7, "nhk-all-7.jpg", html=True)

In [ ]:
notes = " https://geneasyura.github.io/cov19-hm/nhk-7-8.html "
tweet_with_image(tw, "docs/images/nhk-all-7.jpg", twbody7 + today_str + notes)

In [ ]:
df_group_by_w['Prefectures'] = ''
for index, row in df_group_by_w.iterrows():
    df_group_by_w.at[index, 'Prefectures'] = en_prefs['e'][row['都道府県コード'] -1]

In [ ]:
fig7 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_group_by_w, x_start=k.ts, x_end='p1w', y='Prefectures', color=k.case,
    color_continuous_scale='Rainbow', labels={k.case:'cpm'})
twbody7 = "Japan Weekly new confirmed COVID-19 cases / million people"
update_layout(fig7, twbody7, "")
fig7.update_layout(
    width=800, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig7.update_yaxes(autorange="reversed", side="right")
fig7.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.08, 
        tickfont=dict(color="white"),
        ticklabelposition="outside", ticks="inside"))
save_plotly_in_en(fig7, "nhk-all-7.jpg")

In [ ]:
tweet_with_image(tw, "docs/images/en/nhk-all-7.jpg", twbody7 + " (updated on " + today_str + ")")

In [ ]:
hyogo_death_20210518 = df_group_by_w[
    (df_group_by_w[k.pref_name] == '兵庫県') & 
    (df_group_by_w['日付'] == pd.to_datetime('2021-05-24'))]

In [ ]:
hyogo_death_20210518 = hyogo_death_20210518[k.death].values[0]
hyogo_death_20210518
wks = 9

In [ ]:
df_group_by_w.loc[\
           (df_group_by_w['日付'] == pd.to_datetime('2021-05-24')) & \
           (df_group_by_w[k.pref_name] == '兵庫県'), k.death] = 0

In [ ]:
df_group_by_w.loc[\
           (df_group_by_w['日付'] >= pd.to_datetime('2021-03-29')) & \
           (df_group_by_w['日付'] <= pd.to_datetime('2021-05-24')) & \
           (df_group_by_w[k.pref_name] == '兵庫県'), k.death] += (hyogo_death_20210518 / wks)

In [ ]:
fig8 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_group_by_w, x_start=k.ts, x_end='p1w', y=k.pref_name, color=k.death,
    color_continuous_scale='Rainbow', labels={k.death:'dpm'})
twbody8 = "全国 新型コロナ 都道府県別 週毎 死者数"
update_layout(fig8, twbody8, today_str)
fig8.update_layout(
    width=730, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig8.update_yaxes(autorange="reversed", side="right")
fig8.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
show_and_save_plotly(fig8, "nhk-all-8.jpg", html=True)

In [ ]:
tw_msg = twbody8 + today_str \
    + "\n※兵庫県2021/5/18発表の129人は3/29～5/24にdpm平均値として加算\n" + notes
tweet_with_image(tw, "docs/images/nhk-all-8.jpg", tw_msg)

In [ ]:
fig8 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_group_by_w, x_start=k.ts, x_end='p1w', y='Prefectures', color=k.death,
    color_continuous_scale='Rainbow', labels={k.death:'dpm'})
twbody8 = "Japan Weekly new confirmed COVID-19 deaths / million people"
update_layout(fig8, twbody8, today_str)
fig8.update_layout(
    width=730, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig8.update_yaxes(autorange="reversed", side="right")
fig8.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
save_plotly_in_en(fig8, "nhk-all-8.jpg")

In [ ]:
tweet_with_image(tw, "docs/images/en/nhk-all-8.jpg", twbody8 + " (updated on " + today_str + ")")

In [ ]:
# ppm で正規化する
df_ppm = df.copy(deep=True) 
df_ppm = df_ppm[ df_ppm[k.ts] >= pd.to_datetime("2021-06-01")]
df_ppm = df_ppm.astype({k.case: float})
df_ppm = df_ppm.astype({k.death: float})
ppm = 1000000
for index, row in df_ppm.iterrows():
    pname = row[k.pref_name].rstrip('県').rstrip('都').rstrip('府')
    df_ppm.at[index, k.case] = (float(row[k.case]) / ppops[pname]) * ppm
    df_ppm.at[index, k.death] = (float(row[k.death]) / ppops[pname]) * ppm
df_ppm = df_ppm.drop(k.total_case, axis=1)
df_ppm = df_ppm.drop(k.total_death, axis=1)

In [ ]:
df_ppm['p1w'] = df_ppm[k.ts] + pd.Timedelta(hours=23)
#df_ppm

In [ ]:
fig9 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_ppm,
    x_start=k.ts, x_end='p1w',
    y=k.pref_name, color=k.case,
    color_continuous_scale='Rainbow', labels={k.case:'cpm'})
twbody9 = "全国 新型コロナ 都道府県別 感染者数 6/1～"
update_layout(fig9, twbody9, today_str)
fig9.update_layout(
    width=700, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig9.update_yaxes(autorange="reversed", side="right")
fig9.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
show_and_save_plotly(fig9, "nhk-all-9.jpg", html=True)

In [ ]:
print(twbody9 + today_str)
notes = " https://geneasyura.github.io/cov19-hm/nhk-9-10.html "
tweet_with_image(tw, "docs/images/nhk-all-9.jpg", twbody9 + today_str + notes)

In [ ]:
df_ppm['Prefectures'] = ''
for index, row in df_ppm.iterrows():
    df_ppm.at[index, 'Prefectures'] = en_prefs['e'][row['都道府県コード'] -1]

In [ ]:
fig9 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_ppm,
    x_start=k.ts, x_end='p1w',
    y="Prefectures", color=k.case,
    color_continuous_scale='Rainbow', labels={k.case:'cpm'})
twbody9 = "Japan Daily new confirmed COVID-19 cases per million people"
update_layout(fig9, twbody9, today_str)
fig9.update_layout(
    width=710, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig9.update_yaxes(autorange="reversed", side="right")
fig9.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
save_plotly_in_en(fig9, "nhk-all-9.jpg")

In [ ]:
tweet_with_image(tw, "docs/images/en/nhk-all-9.jpg", twbody9 + today_str)

In [ ]:
#hyogo_death_20210518 = df_ppm[df_ppm['日付'] == pd.to_datetime('2021-05-18')]
#hyogo_death_20210518 = hyogo_death_20210518[hyogo_death_20210518[k.pref_name] == '兵庫県']
#hyogo_death_20210518 = hyogo_death_20210518[k.death].values[0]
#hyogo_death_20210518
#td_hyogo = pd.to_datetime('2021-05-17') - pd.to_datetime('2021-03-26')
#hyogo_death_20210518 / td_hyogo.days

In [ ]:
#df_ppm.loc[\
#           (df_ppm['日付'] == pd.to_datetime('2021-05-18')) & \
#           (df_ppm[k.pref_name] == '兵庫県'), k.death] = 0

In [ ]:
#df_ppm.loc[\
#           (df_ppm['日付'] >= pd.to_datetime('2021-03-26')) & \
#           (df_ppm['日付'] <= pd.to_datetime('2021-05-17')) & \
#           (df_ppm[k.pref_name] == '兵庫県'), k.death] += (hyogo_death_20210518 / td_hyogo.days)

In [ ]:
fig10 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_ppm,
    x_start=k.ts, x_end='p1w',
    y=k.pref_name, color=k.death,
    color_continuous_scale='Rainbow', labels={k.death:'dpm'})
twbody10 = "全国 新型コロナ 都道府県別 死者数 6/1～"
update_layout(fig10, twbody10, today_str)
fig10.update_layout(
    width=700, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig10.update_yaxes(autorange="reversed", side="right")
fig10.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
show_and_save_plotly(fig10, "nhk-all-10.jpg", html=True)

In [ ]:
tw_msg = twbody10 + today_str + notes # \
#    + "\n※兵庫県2021/5/18発表の129人は3/26～5/17にdpm平均値として加算\n" + notes
print(tw_msg)
tweet_with_image(tw, "docs/images/nhk-all-10.jpg", tw_msg)

In [ ]:
fig10 = px.timeline(#df_group_by_w[df_group_by_w[k_pref_code] <= 24],
    df_ppm,
    x_start=k.ts, x_end='p1w',
    y="Prefectures", color=k.death,
    color_continuous_scale='Rainbow', labels={k.death:'dpm'})
twbody10 = "Japan Daily new confirmed COVID-19 deaths per million people"
update_layout(fig10, twbody10, today_str)
fig10.update_layout(
    width=720, height=800,
    xaxis=dict(type='date', dtick=2419200000.0, tickformat="%_m/%-d"))
fig10.update_yaxes(autorange="reversed", side="right")
fig10.update_coloraxes(
    showscale=True, 
    colorbar=dict(
        xanchor='left', x=-0.05, 
        tickfont=dict(color="black"),
        ticklabelposition="inside", ticks="inside"))
save_plotly_in_en(fig10, "nhk-all-10.jpg")

In [ ]:
tweet_with_image(tw, "docs/images/en/nhk-all-10.jpg", twbody10 + today_str)